In [0]:
watermark_ts = spark.sql("select * from otc.silver.watermark where table_name='customer' ").select('watermark_ts').collect()[0][0]

eligible_customer_records = spark.sql(f"""
                                      select * from otc.bronze.src_customer
                                      where ingest_ts > '{watermark_ts}'
                                      """)

eligible_customer_records.display()
eligible_customer_records.createOrReplaceTempView("eligible_customer_records")

In [0]:
%sql
describe otc.silver.customer

In [0]:
%sql
SELECT  
    abs(xxhash64(ec.customer_id, ec.ingest_ts)),
    ec.customer_id,
    ec.full_name,
    ec.city,
    ec.ingest_ts,
    NULL,
    TRUE,
    ec.updated_at,
    ec.ingest_ts
    FROM eligible_customer_records as ec 
    LEFT ANTI JOIN otc.silver.customer as c
    ON ec.customer_id = c.customer_id and c.is_current=TRUE


In [0]:
%sql

MERGE INTO otc.silver.customer AS c
USING eligible_customer_records AS ec
ON c.customer_id = ec.customer_id and c.is_current = TRUE
WHEN MATCHED AND (c.city <> ec.city ) THEN
    UPDATE SET 
        c.effective_to = ec.ingest_ts,
        c.is_current = FALSE;


INSERT INTO otc.silver.customer
SELECT  
    abs(xxhash64(ec.customer_id, ec.ingest_ts)),
    ec.customer_id,
    ec.full_name,
    ec.city,
    ec.ingest_ts,
    NULL,
    TRUE,
    ec.updated_at,
    ec.ingest_ts
    FROM eligible_customer_records as ec 
    LEFT ANTI JOIN otc.silver.customer as c
    ON ec.customer_id = c.customer_id and c.is_current=TRUE and ec.city = c.city

In [0]:
%sql
select * from otc.silver.customer